#### Imports 

In [ ]:
from sagemaker import get_execution_role, Session
from sagemaker.huggingface import HuggingFace
import sagemaker
import logging

##### Setup logger 

In [ ]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

In [ ]:
logger.info(f'[Using SageMaker: {sagemaker.__version__}]')

#### Essentials 

In [ ]:
session = Session()
role = get_execution_role()
bucket = session.default_bucket()

### Create a HuggingFace estimator and start a SageMaker training job

In [ ]:
!pygmentize ./src/train.py

##### Define hyperparameters

In [ ]:
hyperparameters={'epochs': 5,
                 'train_batch_size': 32,
                 'model_name':'distilbert-base-uncased'
                 }

Define metric definitions 

In [ ]:
metric_definitions = [
    {"Name": "epoch", "Regex": "epoch.*=\D*(.*?)$"},
    {"Name": "train_runtime", "Regex": "train_runtime.*=\D*(.*?)$"},
    {'Name': 'train_samples_per_second', 'Regex': "train_samples_per_second.*=\D*(.*?)$"},
    {"Name": "train_accuracy", "Regex": "train_accuracy.*=\D*(.*?)$"},
    {"Name": "train_loss", "Regex": "train_loss.*=\D*(.*?)$"},
    {"Name": "eval_accuracy", "Regex": "eval_accuracy.*=\D*(.*?)$"},
    {"Name": "eval_loss", "Regex": "eval_loss.*=\D*(.*?)$"},
    {"Name": "f1", "Regex": "f1.*=\D*(.*?)$"},
    {"Name": "exact_match", "Regex": "exact_match.*=\D*(.*?)$"}
]

Instance configurations

In [ ]:
instance_type = 'ml.p3.16xlarge'
instance_count = 1
volume_size = 200

Create HuggingFace estimator

In [ ]:
huggingface_estimator = HuggingFace(entry_point='train.py',
                                    source_dir='./src',
                                    metric_definitions=metric_definitions,
                                    instance_type=instance_type,
                                    instance_count=instance_count,
                                    volume_size=volume_size,
                                    role=role,
                                    transformers_version='4.6',
                                    pytorch_version='1.7',
                                    py_version='py36',
                                    hyperparameters = hyperparameters)

##### Fit model

In [ ]:
training_input_path = f's3://{bucket}/imdb/train'
test_input_path = f's3://{bucket}/imdb/test'

In [ ]:
%%time

huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path}, logs=False)

##### Retrieve estimator parameters 

In [ ]:
logger.info(f'S3 uri where the trained model is located: {huggingface_estimator.model_data}')

In [ ]:
logger.info(f'Latest training job name for this estimator: {huggingface_estimator.latest_training_job.name}')

#### Deploying the endpoint

In [ ]:
predictor = huggingface_estimator.deploy(1, 'ml.g4dn.xlarge')

Make inferene using the deployed sentiment classifier model

In [ ]:
sentiment_input= {"inputs": "I love using the new Inference DLC."}

In [ ]:
response = predictor.predict(sentiment_input)
response